<a href="https://colab.research.google.com/github/junjy007/UTS_ML2019_Main/blob/master/NB02_Practical_Data_Skills_ArrayOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 Practical Skill 1 -- Array Operations

## 5.1 Brief of Data Wrangling Tools with Discussion on When to Use Which

### 5.1.1 Small arrays and tables

Arrays are a very generic representation of the data. When an array is 2D,  it is often called a table. Usually, each row of a table represent one data sample, and each column of the table corresponds to one attribute of the data. Table data is perhaps the most the common simplest format of data. if all the values in the table are numerical,  and especially when we want to perform computations on the table a `numpy` array (see next subsection for details) is a convenient choice.

On the other hand, a pandas dataframe should be considered if
- the types of the values in the table are different,  or
- we needed to deal with issues such as missing values,  or
- It is convenient in the analysis to refer to the meaning of the attributes,  such as instead of being called “attribute-2”,  we want to call the attribute “petal_length”.

An example of table data is the basic machine learning dataset "Iris".

Note in the following example, `seaborn` is an accompany library facilitating quick prototyping and visualisation / inspection of `DataFrames` of the `pandas` library.

In [0]:
import seaborn as sns

iris = sns.load_dataset('iris')
iris.head()

Alternatively, if we manage the data in numpy arrays

In [0]:
from sklearn.datasets import load_iris

iris_np = load_iris()
print(iris_np.data.shape)
print(iris_np.data[:5])
print(iris_np.target[:5])

### 5.1.2 Homogeneous Arrays

When the data contains large amount of attributes, those attributes are organised in a spatially regular and meaningful way, then we use typical numerical multidimensional arrays. 

The numerical computaiton library `numpy` provides convenient interfaces to efficient computation tools of manipulating arrays. Check the [online document][numpy-doc] for more details.

[numpy-doc]:https://www.numpy.org/

> Numpy has a sister library, `scipy`, which contains higher level (overlapping to an extent) functions. E.g. `numpy` performs linear algebric operators, e.g. compute $C$ in $A \times B \rightarrow C$, where $A$  and $B$ are matrices or vectors, while `scipy` can do the inverse computation, i.e. finding out $A$, where $A \times B \rightarrow C$)


#### Example of image pixels

In [0]:
import numpy as np
from sklearn.datasets import load_sample_images
ims = load_sample_images()
print(ims.images[0].shape, ims.images[1].shape)
images = np.stack(ims.images)
print(images.shape)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(images[0])
plt.show()
plt.imshow(images[1])
plt.show()

Let's apply a partial blue-filter and a total red filter to the two images.

In [0]:
images[0][100:150, 100:500, 0:2] = 0
plt.imshow(images[0])
plt.show()
images[1][:, :, 1:] = 0
plt.imshow(images[1])
plt.show()

## 5.2 Advanced Array Operations

### 5.2.1 Indexing

The following code is a simple visualiser.

In [0]:
def show_array(a):
    if a.ndim == 1: 
        a = a[:, np.newaxis]
    fig, axes = plt.subplots(1)
    fig.set_size_inches(a.shape[1]/2, a.shape[0]/2)
    axes.imshow(a)
    axes.set_xticks(np.arange(a.shape[1]))
    axes.set_yticks(np.arange(a.shape[0]))

Study the following example, and perform the following exercise.

In [0]:
a = np.zeros((10, 1))
a[5:8]=1
show_array(a)

a = np.zeros((10, 1))
a[3:8:2]=1
show_array(a)

__EXERCISE__

In [0]:
# Replace ... to produce (1) and (2) (in two experiments)
a = np.zeros((10, 1))
a[...]=1
show_array(a)

In [0]:
# Replace ... to produce (3) and (4) (in two experiments)
a = np.zeros((10, 10))
a[1::3, ::2]=1
show_array(a)

In [0]:
# Consider the `im` array
ims = load_sample_images()
im = ims.images[0][:,:, 0]/255
plt.imshow(im, cmap="gray")
plt.show()

# Explain what you see in im_h
im_h = im[:, 1:] - im[:, :-1]
plt.imshow(im_h, cmap="gray", vmax=1.0, vmin=-1.0)
plt.show()

### 5.2.2 Reducing and Broadcasting

Reducing refers to some operation that summarise the values along a dimension, and store the result in a new array.  The new array will have one less dimension than the original one has. The summarisation operation include some often encountered ones such as taking the mean value or finding the median value or taking the minimum or maximum.

In [0]:
# E.g. get the mean of each attribute of Iris Data
from sklearn.datasets import load_iris

iris_np = load_iris()
iris_attr_mean = iris_np.data.mean(axis=0)
print(iris_attr_mean)

__EXERCISE__

Compute the max and min value of each attribute of the Iris dataset.

Broadcasting is an easy way to specify element-wise computation between two arrays.  In a sense, the correspondence between a large array and a small array in broadcasting mirrors the relationship between the two arrays in reducing. Please check the document or a [tutorial][bd-tut] for more detials.

[bd-tut]:https://www.tutorialspoint.com/numpy/numpy_broadcasting.htm

In [0]:
# E.g. to take off the minimum value of each attribute from each data sample in Iris, so in the processed data 
# the minimum value of each attribute is 0.

iris_np_zero_min = iris_np.data - iris_np.data.min(axis=0)

In [0]:
plt.boxplot(iris_np_zero_min)
plt.xlabel("Attributes")
plt.show() # Note the absolute minimum value of each attribute.

__EXERCISE__

Normalise the iris data so that each attribute is between $[0, 1]$.

HINT: the range is `max - min`.

# Real-world Data Wrangling Example 1: Sea Surface Temperature Records

In [0]:
try:
    import netCDF4
except:
    !pip install netcdf4
    
try:
    import plotly
    major_plotly_version = int(plotly.__version__.split('.')[0])
    assert major_plotly_version >= 4
except:
    !pip install plotly==4.1.0
    
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import xarray as xr     

In this example, we practice our skills to manipulate large complex array data. We consider the global sea surface temperature (SST) records over the past 100 years. The data consists of measurements conducted by various devices including commercial/military vehicles, satellites, etc. See [Wikipedia][sst] document for further information. The data we will use is from [NOAA/ERSL/PSD][esrl] (The US National Oceanic and Atmospheric Administration Earth System Research Laboratory's Physical Sciences Division). Data source: [link](https://www.esrl.noaa.gov/psd/data/gridded/).

The data is in NetCDF format. They are essentially high-dimensional arrays. However, the dimensions are of physical significance (e.g. longitudes/longitudes/years). We will use the `xarray` library to process the arrays. At the backend, `xarray` uses `netcdf` library to access the data.

[esrl]:https://www.esrl.noaa.gov/psd/ 
[sst]:https://en.wikipedia.org/wiki/Sea_surface_temperature

In [0]:
# Duplicate for convenient reference not need to execute
try:
    import netCDF4
except:
    !pip install netcdf4
import xarray as xr    

## Prepare Data

It is a good habit to maintain the data and the code in an ordered away in the file system.  It allows you to use cloud computing facilities (where you may not be able to store data for long) and make it easier to back up all your work.

In the following code cell, we try to open the dataset file at a designated location.  If the file does not exist, we will download the raw data.  If the location does not exist, we will create the folders accordingly.

In [0]:
from pathlib import Path
DATA_URL = 'ftp://ftp.cdc.noaa.gov/Datasets/COBE2/sst.mon.mean.nc'
DATA_FILENAME = "sst.mon.mean.nc"

data_dir = Path("./data/global-sst")
# Create dir structure if not existing
data_dir.mkdir(parents=True, exist_ok=True) 
filepath = data_dir / DATA_FILENAME

Check if the dataset file exists, if not, download the file.

HINT: If used often, the file is better stored in your own GoogleDrive. To access your GoogleDrive files, you can checkout a code-snippet provided by colab that "mounts" the drive (then you can view it as a local folder). 

In [0]:
if not filepath.exists():
    import urllib.request
    urllib.request.urlretrieve(DATA_URL, str(filepath))
filepath = str(filepath.absolute())    

Now we are ready to load the multi-dimensional array of SST data.

In [0]:
data =xr.open_dataset(filepath)

Now we have `data` -- the object returned by opening the dataset. So we can simply let the Notebook to “print out” the object to get some basic information about it. 

## Initial Exam

In [0]:
data

From the brief description we can figure out that the essential information of this data set is stored in: 
```
Data variables:
    sst      (time, lat, lon) float32 ...
```
So let us further examine this sub-field of the `data` object.

In [0]:
data.sst

We can consider `data.sst` as a normal three dimensional array (similar to `numpy.ndarray`), check its shape and take one slice to visualise.

In [0]:
print("SST data shape", data.sst.shape)
print("The slice of SST data of time-dimension[0] shape:", data.sst[0].shape)
print("Visualise the slice as an image")
plt.imshow(data.sst[0])
plt.show()

Well, the figure looks ... oceanic. But the dataset provides more information about the axes. Let us investigate.

The `xarray` library wraps the data in an object `xarray.DataArray`, which provides us with a more meaningful way of accessing the data by referring to the axes concerning their semantics.

Let us check the dimension of `time`:

In [0]:
data.sst.time

Interestingly, this dimension itself is represented using another `DataArray` object.  We notice its data type (`dtype`) is `datetime64[ns]`, which is a numpy class representing a precise moment.  So let us try to use a time object to index the array (The object was new to me as well,  I just googled using the keywords "numpy datetime64"). 

Note the `squeeze` removes singular dimensions from the result slice. Check `numpy.squeeze` for more details.

In [0]:
day = np.datetime64('1980-12-01')
sst_day = data.sst[data.sst.time == day].squeeze()
plt.imshow(sst_day.values)
plt.show()

Since the sea surface temperature data are measurements on a grid, sometimes it is useful to create coordinate grids for the individual measurements. E.g. the DataArray contains

| .       |    Logitudes| 0| 1| 2| 3| 4| 
|-|-|-|-|-|-|-|
|Latitudes|            .| .| .| .| .| .|
|0|.|$v_{0,0}$|v|v|v|v|
|1|.|v|v|v|v|$v_{1,4}$|

For some convenience, we want to represent the data in one long table

|Latitudes|Logitudes|Value|
|-|-|-|
|0|0|$v_{0, 0}$|
|0|1|$v_{0, 1}$|
...


The table has the same number of rows as the number of total elements in the original 2D array.

In `numpy`, the coordinates can be constructed using `meshgrid`, please google and try.
```python
xx, yy = np.meshgrid(np.array([0, 1, 2]), np.array([99, 100]))
print("x-grid")
print(xx)
print("y-grid")
print(yy)
print("coordinate-grid")
print(np.stack((xx.flatten(), yy.flatten())))
```

As to `xarray.DataArray`, the object provides a direct method to generate a table with coordinate grids via its `to_dataframe()` function. Note the generated frame will have "multiple index" (working similary as the latitudes and logitudes as in the original data), we need to `reset_index()` to have the `lat` and `lon` as independent columns.

In [0]:
sst_day_df = sst_day[::3, ::3].to_dataframe() #::3, take measure 
# sample every 3 locations, to reduce computational demand for visualisation
sst_day_df.reset_index(inplace=True)

We can use plotly to show interactive plot of the SST data.

In [0]:
DRAW_EXPRESS = False
if DRAW_EXPRESS:
    # px.set_mapbox_access_token("pk.eyJ1IjoianVuankwMDciLCJhIjoiY2p6aWs5bDZxMDA1ZDNjczM1cmFiMjc5MCJ9.ixcf00oF_QL5eg44OP9fuQ")
    fig = px.scatter_geo(sst_day_df, lat="lat", lon="lon", color="sst", 
                         color_continuous_scale=px.colors.sequential.Viridis)
    fig.show()
else:
    # More control. But verbose coding.
    fig_geo_data = go.Scattergeo(
        lat=sst_day_df.lat, 
        lon=sst_day_df.lon, 
        text=["sst={:.2f}".format(s_) for s_ in sst_day_df.sst],
        marker=dict(color=sst_day_df.sst, colorscale="Viridis"),
        name="Global Sea Surface Temperature in Dec 1980")
    fig = go.Figure(data=fig_geo_data)
    fig.show()



## Investigate an Area of Interest

Partially to practice array operation skills and partially to investigate an idea, let us extract the SST from a tropical sea area. We construct boolean indexes to take SST values from east tropical pacific ocean (you can try your own idea, e.g. check the "Nino 3.4 Region" related to [El Nino][elnino]).

[elnino]:https://en.wikipedia.org/wiki/El_Ni%C3%B1o

In [0]:
# Specify the boundaries of the area
lat_north = 10.5
lat_south = -10.5
lon_west = 230.5
lon_east = 260.5

Indexing by boolean tests works similarily in mainstream array tools. See this [document][adv-arr-index] of numpy.

[adv-arr-index]:https://docs.scipy.org/doc/numpy-1.13.0/user/basics.indexing.html

In [0]:
lat_index = (lat_south <= data.sst.lat) * (data.sst.lat <= lat_north)
lon_index = (lon_west <= data.sst.lon) * (data.sst.lon <= lon_east)
interested_sea_area = data.sst[:, lat_index, lon_index]
print(interested_sea_area)

The brief information shows the area is a cut-off block from the global SST data of 22 latitudes and 31 longitudes.

In [0]:
# Heatmap is another intuitive visualisation for array data.
fig = go.Figure(data=[go.Heatmap(x=interested_sea_area.lon,
                                 y=interested_sea_area.lat,
                                 z=interested_sea_area[0,:,:])])
fig.show()

## Examine Trend in Time

Let us examine if there is any time-related trend in the average temperature of the sea area. We need to compute the average SST value for the entire range of longitudes and latitudes. The computation needs to be done for every time spot. This corresponds to the "reduce" operation introduced above. The "reducing" applies to two dimensions: "lon" for longitudes and "lat" for latitudes. After the operation, we will arrive at a series (1-D) of average SST values, each corresponding to one time of record.

In [0]:
interested_sea_area_sst_mean = interested_sea_area.mean(dim=["lon", "lat"])
print(interested_sea_area_sst_mean)

In [0]:
scatter_data = go.Scatter(
    x=interested_sea_area_sst_mean.time.to_series(), # so the time is human readable
    y=interested_sea_area_sst_mean,
    mode="markers+lines")
fig = go.Figure(data=[scatter_data])
fig.show()

The data is obviously periodic -- the temperature rises and falls through a year. Let's take a look at a particular month, say December.

Also for practice, lets work with `pandas.DataFrame`. Following an [example][pandas-datetime], we can add auxillary-columns to the dataframe to facilitate selection. Note the `apply` operation represents an important design pattern "mapping/transforming", where we give a list of object and a processing function. The data manager (here is the `DataFrame`) is responsible to pass each object in the list sequentially to the function to process.

[pandas-datetime]:https://medium.com/datadriveninvestor/how-to-work-with-dates-in-pandas-like-a-pro-a84055a4819d

In [0]:
sst_area_mean_df = interested_sea_area_sst_mean.to_dataframe()
sst_area_mean_df.reset_index(inplace=True)

sst_area_mean_df["month"] = sst_area_mean_df["time"].apply(lambda x:x.month) #
sst_area_mean_df["year"] = sst_area_mean_df["time"].apply(lambda x:x.year) #
sst_area_mean_df.head() # Now we have access to year and month

For exploratory purposes, I would adopt a quick dirty trick. We just take one particular month from each year and examine the trend of the average SST in the area of this month.

In [0]:
sst_area_mean12_df = sst_area_mean_df.loc[sst_area_mean_df["month"]==12]

Now let's visualise the data.

In [0]:
DRAW_EXPRESS = True
if DRAW_EXPRESS:
    fig = px.scatter(sst_area_mean12_df, x="year", y="sst", trendline="ols", range_x=[1849, 2020])
    
else:
    fig = go.Figure(data=[go.Scatter(
        x=sst_area_mean12_df["year"], 
        y=sst_area_mean12_df["sst"],
        mode="markers+lines")]
    )
fig.show()

Any trend? How does the ocean affect us?

# Real-world Data Wrangling Example 2: World Food Production Data

The [data][unfao-crop-product-data] is from the United Nations Food and Agriculture Organisation ([UN FAO][unfao]). In this example, we will look at the [crop production][unfao-qc]. UNFAO publishes a wide range of data related to food, agriculture, as well as social development and human wellbeing. See their [document][unfao-crop-product-spec] for more information.

[unfao-crop-product-data]:http://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_E_All_Data.zip
[unfao]:http://www.fao.org/faostat/en/#home
[unfao-qc]:http://www.fao.org/faostat/en/#data/QC
[unfao-crop-product-spec]:http://fenixservices.fao.org/faostat/static/documents/QC/QC_methodology_e.pdf

## Prepare Data

At above, we fetch the data from the Internet if not existing.

In [0]:
# ** Food Data File **
FAOSTAT_DATA_URL = "http://fenixservices.fao.org/faostat/static/bulkdownloads/" +\
    "Production_Crops_E_All_Data_(Normalized).zip"
DATA_FILENAME = "Production_Crops_E_All_Data_(Normalized).zip"
data_dir = Path("./data/food-supply")
# Create dir structure if not existing
data_dir.mkdir(parents=True, exist_ok=True) 
filepath = data_dir / DATA_FILENAME

if not filepath.exists():
    import urllib.request
    urllib.request.urlretrieve(FAOSTAT_DATA_URL, str(filepath))    
filepath = str(filepath.absolute()) 

# ** Country Code File ** (for visualisation)
# Note the github raw file link is DIFFerent from the file-view link, see 
# the link below for more information:
# https://stackoverflow.com/questions/4604663/download-single-files-from-github
ISO_COUNTRY_CODE_CSV_URL = \
    "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
ISO_COUNTRY_CODE_CSV_FILENAME = "ISO-3166.csv"

iso_country_code_csv_filename = data_dir / ISO_COUNTRY_CODE_CSV_FILENAME
if not iso_country_code_csv_filename.exists():
    import urllib.request
    urllib.request.urlretrieve(ISO_COUNTRY_CODE_CSV_URL, str(iso_country_code_csv_filename))
iso_country_code_csv_filename = str(iso_country_code_csv_filename.absolute())
iso_country_df = pd.read_csv(iso_country_code_csv_filename)

## Initial Explore

The FAO data is a table. We read it using `pandas.DataFrame`. `pandas` can handle compressed CSV file. However, the table contains characters not encoded in 'utf-8', we need to specify the encoding. 

Note for exploratory manipulation, I found it is handy to keep a cheatsheet of the data toolbox in reach. In this example, consider [this cheatsheet][pandas-cheatsheet].

[pandas-cheatsheet]:https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [0]:
df = pd.read_csv(filepath,
    compression='zip', encoding = "ISO-8859-1")

First glance:

In [0]:
df.head() # What columns does it have? How the data samples look like?

There are about 10 informative columns. What is the size of the data?

In [0]:
len(df)

That is a lot.  Let's narrow our focus for now. E.g. consider some `Item` in particular. However, even if we only look at the `Item` column, there still be 2M+ elements:

In [0]:
df['Item']

But it seems there are a lot of repetitions. So the natural idea is to check what are the unique values in this list.

In [0]:
df['Item'].unique()

This looks much more manageable.

Note the `unique` keyword is universal in many toolboxes. Usually, a library also provides a more convenient function that checks both the unique values and how many data samples having each value.

In [0]:
df['Item'].value_counts()

To begin with, let us consider one item, "Wheat" ("Bananas" and "Vegetables Primary" may also be good choices for experiments).

In [0]:
wheat_data = df.loc[df['Item'] == "Wheat"]

In [0]:
wheat_data.head(5)

It looks the table also contains different aspects relating to the production of wheat. As above, we consider the unique values of the column “Element”.

In [0]:
wheat_data['Element'].value_counts()

As above, we consider the unique values of the column “Element”. Now we can form clearer ideas about the table. In the next, we consider the “Yield” (it seems to be the yield of unit land area).

In [0]:
wheat_yield = wheat_data.loc[wheat_data["Element"] == "Yield"]

In [0]:
wheat_yield.head()

Let us check the yield of a few areas in the world. Note the query to the `DataFrame` below. You can use `wheat_yield["Area"].unique()` to find the area names to form your queries. Please refer to [pandas document][qry-doc] for more information about queries in `pandas`.

[qry-doc]:https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-query

In [0]:
px.scatter(wheat_yield.query('Area == ["Australia", "India", "China", "United States of America"]'), 
           x="Year", y="Value", color="Area", trendline="ols")

Well the trend looks encouraging enough to deserve a movie! :P 

In [0]:
px.scatter(wheat_yield.query('Area == ["Australia", "India", "China", "United States of America"]'), 
           x="Year", y="Value", size="Value", color="Area",
           animation_frame="Year", animation_group="Area", range_x=[1950, 2020], range_y=[0, 60000])

But there seems to be oscillations in the growth, which may deserve further investigation.

## Visualise and Examine the Aspect of Interest

Let us picture the yield of wheat globally,  which both gives us first impression about the trend and looks nice for report.

To generate the global picture with geographical information, we would need a list of the country code.   We have downloaded the code list with country names from the Internet. Now we want to merge this table to the food production table.

In [0]:
# We care only about name-code correspondence in the area-code table.
# And we want the country-name-column are referred to consistently as "Area" 
# in both tables.
iso_info_df = iso_country_df[["name", "alpha-3"]].rename(columns={"name": "Area", "alpha-3": "AreaAlphaCode"})
iso_info_df.head()

Now we merge the two tables with respect to the column of country names.

In [0]:
wheat_yield_cc = pd.merge(wheat_yield, iso_info_df, on="Area") # Get Area Code for Visualisation
wheat_yield_cc.head()

In [0]:
px.scatter_geo(wheat_yield_cc, locations="AreaAlphaCode", size="Value", animation_frame="Year")

The bubbles seem to be breathing. Let's try to find out if the physical process on the earth has something to do we our food.

## Explore Links between two Processes

Let us combine the food data with the SST data. Note we need to rename the column of "year" in SST data to "Year". The match must be exact.

In [0]:
food_ocean = pd.merge(wheat_yield_cc, sst_area_mean12_df.rename(columns={"year": "Year"}), on="Year")
food_ocean.head()

The first step is to compute the increase of the production from one year to the next.

In [0]:
dfs = []
for cc in ["AUS", "CHN", "IND", "USA"]:
    df = food_ocean.loc[food_ocean.AreaAlphaCode == cc]
    df["Growth"] = df.Value.shift(-1) - df.Value # check how shift does to data frames
    dfs.append(df)
    
g_df = pd.concat(dfs)
g_df['SSTDevi'] = g_df.sst - g_df.sst.mean()

In [0]:
px.scatter(g_df, x="Growth", y="SSTDevi", size="Value", color="Area", animation_frame="Year",
          range_y=[-3, +3], range_x=[-10000, 10000])

More investigation is needed.

Check the following reference for more information

http://www.fao.org/3/y4011e/y4011e04.htm